In [1]:
import geopandas as gpd
import ipympl
from ipywidgets import AppLayout, Dropdown
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point

%matplotlib widget
plt.ioff()

In [2]:
# Import data. TODO: Outsource to one location to increase Voila serving process.
df_enc = pd.read_csv('./data/data_preprocessed/features.csv')
df = pd.read_csv('./data/data_raw/nyc_trees/nyc_tree_census_2015.csv.gz', compression='gzip')
nyc = gpd.read_file('./data/data_raw/nyc/nyc_geo.shp')
nyc_streets = gpd.read_file('./data/data_raw/nyc_streets/geo_export_f311eef5-5969-43eb-a781-39fa54d09b70.shp')

# Bring data into right format.
trees = gpd.GeoDataFrame(df, geometry=[Point(coordinates) for coordinates in zip(df['longitude'], df['latitude'])])
trees.crs = {'init': 'epsg:4326', 'no_defs': True}
nyc = nyc.to_crs(epsg=4326)
nyc.crs = {'init': 'epsg:4326', 'no_defs': True}
trees_grouped = trees.groupby('borough').count()['tree_id']
#trees_per_borough = nyc.merge(trees_grouped, left_on='bname', right_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [3]:
# Specifiy dropdown menu for displaying trees.
sel_boroughs = Dropdown(options=[('All street trees', 1),
                                 ('Healthy street trees', 2),
                                 ('Dead street trees', 3),
                                 ('Ok street trees', 4)],
                        value=1,
                        description='Display:')
sel_boroughs.layout.margin = '0px 30% 0px 30%'
sel_boroughs.layout.width = '40%'
sel_boroughs.layout.padding = '2.0'

fig, ax = plt.subplots()
fig.canvas.header_visible = False
plt.title('Street trees in New York',
           fontdict={'fontsize': 25},
           pad=2.5)
fig.canvas.layout.min_height = '400px'
ax.axis('off')

# Plot NYC boroughs, streets, and trees.
nyc.plot(alpha=0.7, ax=ax, color='grey', edgecolor='k')
for borough in nyc.itertuples(index=True, name='bname'):
    plt.text(borough.geometry.centroid.x,
             borough.geometry.centroid.y,
             borough.bname, 
             bbox=dict(boxstyle='square, pad=0.3',
                       edgecolor='k',
                       facecolor='white'))

# Plot streets for additional context.
nyc_streets.plot(ax=ax, alpha=0.2, color='grey')
tree_visual = trees.plot(ax=ax, alpha=1, color='white', edgecolor='white', markersize=1.4)

def update_view(change, tree_visual=tree_visual):
    """
    Updates the figure drawn above.
    """
    
    if change.new == 2:
        tree_visual = trees.loc[df_enc.loc[df_enc['health_Good']==1].index]
        tree_visual.plot(ax=ax, color='green', edgecolor='green', markersize=1.4)
    elif change.new == 3:
        tree_visual = trees.loc[df_enc.loc[df_enc['health_Dead|Stump']==1].index]
        tree_visual.plot(ax=ax, color='red', edgecolor='red', markersize=1.4)
    elif change.new == 4:
        tree_visual = trees.loc[df_enc.loc[df_enc['health_Poor|Fair']==1].index]
        tree_visual.plot(ax=ax, color='orange', edgecolor='orange', markersize=1.4)
    else:
        tree_visual = trees.plot(ax=ax, alpha=1, color='white', edgecolor='white', markersize=1.4)
        fig.canvas.draw()
    
    return tree_visual
    
sel_boroughs.observe(update_view, names='value')

AppLayout(center=fig.canvas,
          footer=sel_boroughs,
          pane_heights=[0, 10, 2])

AppLayout(children=(Dropdown(description='Display:', layout=Layout(grid_area='footer', margin='0px 30% 0px 30%…